In [60]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
import traceback


In [61]:
# Load environment variables
load_dotenv()
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")




In [62]:

from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",  # ✅ required
    temperature=0.5,
    max_new_tokens=1024,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)



In [63]:
# Sample JSON format for MCQs
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}




In [64]:
# Prompt for quiz generation
TEMPLATE = """
Text: {text}

You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in a {tone} tone. 

Make sure the questions are not repeated and all questions are directly based on the text.
Ensure the format matches the RESPONSE_JSON below. Create exactly {number} MCQs.

### RESPONSE_JSON
{response_json}
"""




In [65]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)




In [66]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)




In [67]:
# Prompt for evaluating the quiz quality
TEMPLATE2 = """
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students,\
you need to evaluate the complexity of the quiz and provide a short 50-word analysis.

If the quiz is too complex or not well-suited for the students, update the questions and modify the tone accordingly.

Quiz_MCQs:
{quiz}

Provide your expert review of the quiz:
"""




In [68]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)


In [69]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [70]:
# Combined chain
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [71]:
# Read input content
file_path = r"C:\Users\sridh\mcqgen\data.txt"
with open(file_path, 'r') as file:
    TEXT = file.read()




In [72]:
# Configuration
NUMBER = 5
SUBJECT = "biology"
TONE = "simple"


In [73]:
# Run the chain
response = generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)






> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defin

c:\Users\sridh\mcqgen\env\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert English grammarian and writer. Given a Multiple Choice Quiz for biology students,you need to evaluate the complexity of the quiz and provide a short 50-word analysis.

If the quiz is too complex or not well-suited for the students, update the questions and modify the tone accordingly.

Quiz_MCQs:

### MCQs
{"1": {"mcq": "Which scientific field studies the fundamental biological processes that occur in living organisms?", "options": {"a": "Physics", "b": "Chemistry", "c": "Biology", "d": "Mathematics"}, "correct": "c"}, "2": {"mcq": "Which major theme of biology explains the unity and diversity of life?", "options": {"a": "Cellular structure and function", "b": "Evolution", "c": "Metabolism", "d": "Reproduction"}, "correct": "b"}, "3": {"mcq": "Which of the following is not a major theme of biology?", "options": {"a": "Cellular structure and function", "b": "Energy processing", "c": "Regula

c:\Users\sridh\mcqgen\env\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



> Finished chain.

> Finished chain.


In [74]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [79]:
import re

quiz_raw = response.get("quiz", "")

# Try to find the first valid JSON-looking object
match = re.search(r"({.*})", quiz_raw, re.DOTALL)
if match:
    try:
        quiz = json.loads(match.group(1))
    except Exception as e:
        print("❌ JSON loading failed:", e)
        quiz = {}
else:
    print("❌ No JSON object found in quiz response")
    quiz = {}




In [80]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [f"{option}: {option_value}" for option, option_value in value["options"].items()]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})



In [81]:
# Save to CSV
quiz_df = pd.DataFrame(quiz_table_data)
quiz_df.to_csv("mcq_output.csv", index=False)


# Print review
print("\nQuiz Review:\n", response.get("review"))


Quiz Review:
 
The quiz is well-structured and appropriate for biology students. The questions are clear, concise, and cover essential topics in biology. The use of multiple-choice format makes it easier for students to identify correct answers. However, some students may find question 5 challenging, as it requires knowledge of the domain of Protists, which can be less familiar to some students. Overall, the quiz is suitable for the intended audience and provides a good assessment of their foundational knowledge in biology.
